In [ ]:
import pandas as pd
import pathlib as pl
file = pl.Path("../recieved.csv")

headerNames = ["time", "recieverId", "tempInC", "pressure", "altitude", "humidity", "capacitance"]
recievedDf = pd.read_csv(file, names=headerNames)  # dF means DataFrame, a construct in pandas for tabled data
print(recievedDf)

In [ ]:
# pd.options.plotting.backend = "plotly"
import plotly.express as px
import numpy as np
df1 = recievedDf[recievedDf['recieverId']==1]

df2 = recievedDf[recievedDf['recieverId']==2]

df3 = recievedDf[recievedDf['recieverId']==3]
df3 = df3[np.abs(df3.capacitance - df3.capacitance.mean()) <= (3 * df3.capacitance.std())] #keep only the ones that are within +3 to -3 standard deviations in the column 'Data'.
df3 = df3[~(np.abs(df3.capacitance - df3.capacitance.mean()) > (3 * df3.capacitance.std()))] #or the other way around

df4 = recievedDf[recievedDf['recieverId']==4]

fig1 = px.scatter(df1, x="time", y="tempInC", title="Reciever 1")
fig1.show()
fig2 = px.scatter(df2, x="time", y="tempInC", title="Reviever 2")
fig2.show()
fig3 = px.scatter(df3, x="time", y="tempInC", title="Reviever 3")
fig3.show()
fig4 = px.scatter(df4, x="time", y="tempInC", title="Reviever 4")
fig4.show()

In [ ]:
# MAP Example
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_mapbox(df, lat="centroid_lat", lon="centroid_lon", color="peak_hour", size="car_hours",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10,
                  mapbox_style="carto-positron")
fig.show()